
Голощапова Алла, 306 группа

In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("forestfires.csv")

In [3]:
# # найдем границы 1.5 * IQR по каждому столбцу
# Q1 = data.quantile(0.25)
# Q3 = data.quantile(0.75)
# IQR = Q3 - Q1
# lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

# mask_out = ((data < lower) | (data > upper)).any(axis = 1)

# df = data[~mask_out]
# data

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

categorical_features = ['month', 'day']
df_num = df.drop(categorical_features, axis=1)

encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[categorical_features])

encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out())
encoded_df.reset_index(drop=True, inplace=True)

df_concat = pd.concat([df_num.reset_index(drop=True), encoded_df], axis=1)

X = df_concat.drop(['area'], axis=1)
y = df_concat['area']

X.columns = X.columns.astype(str)
X_train, X_test, y_train, y_test  =  train_test_split(X, y, test_size=0.2, random_state=42)

# # Удаление старых категориальных столбцов и объединение с новыми
# numerical_data = df.select_dtypes(exclude=['object'])
# scaler = MinMaxScaler()
# numerical_data_scaled = pd.DataFrame(scaler.fit_transform(numerical_data), columns=numerical_data.columns)

In [7]:
# numerical_data = X_train #.select_dtypes(exclude=['object'])
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)


y_train_scaled = np.log(y_train+1)
y_test_scaled = np.log(y_test+1)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train_scaled, y_train_scaled)
y_pred = lr.predict(X_test_scaled)
rmse_lr = mean_squared_error(y_test_scaled, y_pred, squared=False)
rmse_lr

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


1.552583611825793

In [9]:
from sklearn.feature_selection import RFECV

rfecv = RFECV(estimator=lr,  scoring='neg_mean_squared_error')
rfecv.fit(X_train, y_train)
selected_features = X.columns[rfecv.support_]

In [10]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.impute import SimpleImputer
lr_selected = LinearRegression()
lr_selected.fit(X_train_scaled[selected_features], y_train_scaled)
y_pred_selected = lr_selected.predict(X_test_scaled[selected_features])
rmse_lr_selected = mean_squared_error(y_test_scaled, y_pred_selected, squared=False)
rmse_lr_selected

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


1.5225185613269037

In [11]:
ridge = Ridge()
ridge.fit(X_train_scaled, y_train_scaled)

lasso = Lasso()
lasso.fit(X_train_scaled, y_train_scaled)

Lasso()

In [12]:
y_pred_ridge = ridge.predict(X_test_scaled)
rmse_ridge = mean_squared_error(y_test_scaled, y_pred_ridge, squared=False)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
y_pred_lasso = lasso.predict(X_test_scaled)
rmse_lasso = mean_squared_error(y_test_scaled, y_pred_lasso, squared=False)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
print("RMSE лин регрессии:", rmse_lr)
print("RMSE с отобранными признаками:", rmse_lr_selected)
print("Ridge :", rmse_ridge)
print("Lasso :", rmse_lasso)

RMSE лин регрессии: 1.552583611825793
RMSE с отобранными признаками: 1.5225185613269037
Ridge : 1.4944271135372813
Lasso : 1.4829780232989842
